# 1 fundamentals

This is a text cell. Start editing!

# AWS Region


AWS has regions all around the world


Names can be used: us-east-1, eu-west-3


A region is a cluster of data centers


Most AWS services are region-scoped


<br>
# AWS Availability Zones


Each region has many availability zones (usually 3, min 2, max is 6)


Example:


ap-southeast-2a


ap-southeast-2b


ap-southeast-2c


Each availability zone (AZ) is one or more discrete data centers with redundant power networking, and connectivity


They are separate from each other, so that they are isolated from disasters


They are connected with high bandwidth, ultra-low latency networking


<br>
# Identity And Access Management (IAM)


## Introduction


Your whole AWS security is there


* Users : usually a physical person
* Groups : Functions(admins, devops) Teams(engineering, design) Contains users
* Roles : Internal usage AWS resources


Root account should never be used (and shared)


Users must be created with proper permissions


IAM is at the center of AWS


Policies are written in JSON (JavaScript Object Notation)


IAM has a global view


Permissions are governed by Policies (JSON)


MFA (Multi Factor Authentication) can be setup


IAM has predefined "managed policies"


We will see IAM policies in details in the future


It's best to give users the minimal amount of permissions they need to perform their job (least privilege principles)


## Federation


Big enterprises usually integrate their own repository of users with IAM


This way, one can login into AWS using their company credentials


Identity Federation uses the SAML standard (Active Directory)


## 101 Brain Dump


One IAM User per PHYSICAL PERSON


One IAM Role per Application


IAM credentials should NEVER BE SHARED


Never, ever, ever, ever, write IAM credentials in code. EVER


Never user the ROOT account except for initial setup


Never use ROOR IAM Credentials


## Multi-factor Authentication MFA


is optional


<br>



# EC2

## What is EC2?

EC2 is one of most popular of AWS offering

It mainly consists in the capability of

* Renting virtual machines (EC2)
* Storing data on virtual drives (EBS)
* Distributing load across machines (ELB)
* Scaling the services using an auto-scaling group (ASG)

Knowing EC2 is fundamental to understand how the Cloud works



# Security Groups


## Introduction


Security Group are the fundamental of network security in AWS


They control how traffic is allowed into or out of our EC2 Machines


It is the most fundamental skill to learn to troubleshoot networking issues


In this lecture, we will learn how to use them to allow, inbound and outbound ports


## Deeper Dive


Security groups are acting as a firewall on EC2 instances


They regulate


* Access to Ports
* Authorised IP rages - IPV4 and IPV6
* Control of inbound network (from other to the instance)
* Control of outbound network (from the instance to the other)


## Good to know

Can be attached to multiple instances

Locked down to a region/VPC combination

Does live outside the EC2 - if traffic is blocked the EC2 instance won't see it

It's good to maintain an separate security group for SSH access

If yous application is not accessible (time out), then it's a security group issue

If your application gives a "connection refused" error, then it's an application error or it's not launched

All inbound traffic is blocked by default

All outbound traffic is authorised by default




# Private vs Public IP (IPv4)

## Introduction

Networking has two sorts of IPs. IPv4 and IPv6

IPv4: 1.160.10.240

IPv6: 3ffe:1900:4545:3:200:f8ff:fe21:67cf

IPv4 is still the most common format used online

IPv6 is newer and solves problems for the Internet of Things (IoT)

IPv4 allows for 3.7 billion different addresses in the public space

IPv4: [0-255].[0-255].[0-255].[0-255]

## Fundamental Differences

Public IP

* Public IP means the machine can be identified on the internet (WWW)
* Must be unique across the whole web (not two machines can have the same public IP)
* Can be geo-located easily

Private IP

* Private IP means the machine can only be identified on a private network only
* The IP must be unique across the private network
* BUT two different private networks (two companies) can have the same IPs
* Machines connect to WWW using a NAT + internet gateway (a proxy)
* Only a specified range of IPs can be used as private IP

## Elastic IPs

When you stop and the start an EC2 instance, it can change its public IP

If you need to have a fixed public IP for your instance, you need an Elastic IP

An Elastic IP is a public IPv4 IP you own as long as you don't delete it

You can attach it to one instance at a time

## Elastic IP

With an Elastic IP address, you can mask the failure of an instance or software by rapidly remapping the address to another instance in your account.

You can only have 5 Elastic IP in your account (you can ask AWS to increase that)

Overall, try to avoid using Elastic IP

* They often reflect poor architectural decisions
* Instead, use a random public IP and register a DNS name to it
* Or, as we will see later, use a Load Balancer and don't use a public IP

## In AWS EC2

By default, your EC2 machine comes with

* A private IP for the internal AWS Network
* A public IP, for the WWW

When we are doing SSH into our EC2 machines

* We can't use a private IP, because we are not in the same network 
* We can only use the public IP

If your machine is stopped and then started, the public IP can change



# EC2 User Data

It is possible to bootstrap our instances using a EC2 User Data script 

Bootstrapping means launching commands when a machine starts

That script is only run once at the instance first start 

EC2 User Data is used to automate boot tasks such as

* Installing updates 
* Installing software
* Downloading common files from the internet 
* Anything you can think of

The EC2 User Data script runs with the root user



# EC2 Instance Launch Types


## Introduction


On Demand Instances: short workload, predictable pricing


Reserved: (MINIMUM 1 Year)<br>* Reserved Instances: long workloads


* Convertible Reserved Instances: long workloads with flexible instances
* Scheduled Reserved Instances: example - every Thursday between 3 and 6 pm


Spot instances: short workloads, for cheap, can lose instances (less reliable)


Dedicated Instances: no other customers will share your hardware


Dedicated Hosts: book an entire physical server, control instance placement


## EC2 On Demand


Pay for what you use (billing per second, after the first minute)


Has the highest cost but no upfront payment


No long term commitment


Recommended for short-term and un-interrupted  workloads, where you can't predict how how the application will behave


## EC2 Reserved Instances


Up to 75% discount compared to On-Demand


Pay upfront for what you use with long term commitment


Reservation period can be 1 or 3 years


Reserved a specified instance type


Recommended for steady state usage applications (think database)


Convertible Reserved Instance


* Can change the EC2 instance type
* Up to 54% discount


Scheduled Reserved Instances


* Launch within time window you reserve
* When you require a fraction of day / week / month


## EC2 Spot Instances


Can get a discount of up to 90% compared to On-Demand


Instances that you can "lose" at any point of the time if your max price is less than the current spot price


The MOST cost efficient instances in AWS


Useful for workloads that are resilient to failure


* Batch jobs
* Data analysis
* Image processing


Not a great for critical jobs or databases


Great combo: Reserved Instances for baseline + On-Demand & Spot for peaks


### EC2 Spot Instance Requests


* Define max spot price and get the instance while current spot price < max<br>
   * The hourly spot price varies based on offer and capacity 
   * If the current spot price > your price you can choose stop or terminate your instance with a 2 minutes grace period


* Other strategy: Spot Block
   * "block" spot instance during a specified time frame (1 to 6 hours) without interruptions
   * In rare situations, the instance may be reclaimed


* Used for batch jobs, data analysis, or workloads that are resilient to failures
* Not great for critical jobs or databases

### Spot Fleets

* Spot Fleets = set of Spot Instances + (optional) On-Demand Instances
* The Spot Fleet will try to meet the target capacity with price constrains 
   * Define posible launch pools: instance type (m5:large), OS, Availability Zone
   * Can have multiple launch pools, so that the fleet can choose
   * Spot fleet stops launching instances when reaching capacity pr max cost


* Strategies to allocate Spot Instances
   * lowestPrice: from te pool with the lowest price (cost optimization, short workload)
   * diversified: distributed across all pools (great for availability, long workloads)
   * capacityOptimized: pool with the optimal capacity for the number of instances

**Spot Fleets allow us to automatically request Spot Instances with lowest price**


## EC2 Dedicated Hosts


Physical dedicated EC2 server for your use


Full control of EC2 Instances placement


Visibility into the underlying sockets / physical cores of the hardware


Allocated for your account for a 3 year period reservation


More expensive


Useful for software that have complicated licensing model (BYOL - Bring Your Own License)


Or for companies that have strong regulatory or compliance needs


## EC2 Dedicated Instances


Instances running on hardware that's dedicated to you


May share hardware with other instances in same account


No control over placement (can move hardware after Stop/Start)


## Which host is right for me?


On demand: coming and staying in resort whenever we like, we pay the full price


Reserved: like planning ahead and if we plan to stay for a long time, we may get a good discount


Spot instances: the hotel allows people to bid for the empty rooms and the highest bidder keeps the rooms. You can get kicked out at any time


Dedicated hosts: we book an entire building of the resort


<br>


# EC2 Instance Type

**R **applications that needs a lot of RAM -in-memory caches

**C** applications that need good CPU - compute / databases

**M** applications that need are balanced (think medium) - general / web app

**I **applications that need good local I/O (instances storage) - databases

**G** applications that need a GPU - video rendering / machine learning

**T2 / T3** burstable instances (up to a capacity)

**T2 / T3** unlimited : unlimited burst

Real-world tip: use https://www.ec2instances.info

## Burstable Instances (T2/T3)

AWS has concept of burstable instances (T2/T3 machines)

Burst means that overall, the instances has OK CPU performance

When the machine needs to process something unexpected (a spike in load for example), it can burst, and CPU can be very good

If the machine burst, it utilizies "burst credits"

If all the credits are gone, the CPU becomes BAD

If the machine stops bursting, credits are accumulated over time

Burstable instances can be amazing to handle unexpected traffic and getting the insurance that it will be handled correctly 

If your instance consistently 

<br>

<br>

